In [1]:
import sys

sys.path.append("/home/jovyan/api")

from nb.debug import (
    setup_debug_session,
    execute_single_processor,
    check_processor_dependencies,
    display_processing_dag,
)

In [2]:
import aiohttp
import os
from typing import Optional


async def upload_test_file(
    filepath: str, queue_processing: bool = False
) -> Optional[str]:
    """Upload a test file without queueing processors"""
    if not os.path.exists(filepath):
        print(f"File not found: {filepath}")
        return None

    async with aiohttp.ClientSession() as session:
        with open(filepath, "rb") as f:
            filename = os.path.basename(filepath)
            data = aiohttp.FormData()
            data.add_field("file", f, filename=filename)

            async with session.post(
                "http://localhost:8000/api/upload",
                data=data,
                params={"queue_processing": queue_processing},
            ) as response:
                if response.status == 200:
                    result = await response.json()
                    return result.get("file_hash")
                else:
                    print(f"Upload failed with status {response.status}")
                    return None




In [5]:
# Upload your test file without queueing processors
filepath = "nb/notes-1.png"  # Replace with actual file path
file_hash = await upload_test_file(filepath, queue_processing=False)

if file_hash:
    print(f"File uploaded successfully. Hash: {file_hash}")
    # Display the full processing DAG
    display_processing_dag(file_hash)
else:
    print("Failed to upload test file")

TypeError: Invalid variable type: value should be str, int or float, got False of type <class 'bool'>

In [3]:
# Setup debug session
run_id, trace = setup_debug_session(file_hash)
print(f"Debug session initialized with run_id: {run_id}")

In [4]:
# Process tables (no dependencies)
try:
    print("Processing tables...")
    check_processor_dependencies("tables", file_hash)
    result_tables = await execute_single_processor(file_hash, "tables", run_id)
    print("Tables processing complete!")
except Exception as e:
    print(f"Error processing tables: {str(e)}")

In [6]:
# Process images (no dependencies)
try:
    print("Processing images...")
    check_processor_dependencies("images", file_hash)
    result_images = await execute_single_processor(file_hash, "images", run_id)
    print("Images processing complete!")
except Exception as e:
    print(f"Error processing images: {str(e)}")

In [7]:
# Process refined (no dependencies)
try:
    print("Processing refined...")
    check_processor_dependencies("refined", file_hash)
    result_refined = await execute_single_processor(file_hash, "refined", run_id)
    print("Refined processing complete!")
except Exception as e:
    print(f"Error processing refined: {str(e)}")

In [8]:
# Process refined_metadata (depends on refined)
try:
    print("Processing refined_metadata...")
    check_processor_dependencies("refined_metadata", file_hash)
    result_refined_metadata = await execute_single_processor(
        file_hash, "refined_metadata", run_id
    )
    print("Refined metadata processing complete!")
except Exception as e:
    print(f"Error processing refined_metadata: {str(e)}")

In [9]:
# Process refined_splitting (depends on refined)
try:
    print("Processing refined_splitting...")
    check_processor_dependencies("refined_splitting", file_hash)
    result_refined_splitting = await execute_single_processor(
        file_hash, "refined_splitting", run_id
    )
    print("Refined splitting processing complete!")
except Exception as e:
    print(f"Error processing refined_splitting: {str(e)}")

In [10]:
# Process lexemes (depends on refined_metadata)
try:
    print("Processing lexemes...")
    check_processor_dependencies("lexemes", file_hash)
    result_lexemes = await execute_single_processor(file_hash, "lexemes", run_id)
    print("Lexemes processing complete!")
except Exception as e:
    print(f"Error processing lexemes: {str(e)}")

In [11]:
# Process citations (depends on lexemes)
try:
    print("Processing citations...")
    check_processor_dependencies("citations", file_hash)
    result_citations = await execute_single_processor(file_hash, "citations", run_id)
    print("Citations processing complete!")
except Exception as e:
    print(f"Error processing citations: {str(e)}")

In [12]:
# Process definitions (depends on citations)
try:
    print("Processing definitions...")
    check_processor_dependencies("definitions", file_hash)
    result_definitions = await execute_single_processor(
        file_hash, "definitions", run_id
    )
    print("Definitions processing complete!")
except Exception as e:
    print(f"Error processing definitions: {str(e)}")